In [397]:
import pandas as pd
import utils

bios = pd.read_json("../../data/congress-bios.json")
currentBios = bios[bios.congresses.apply(lambda x: 118 in x)]
recentBios = bios.loc[
    [
        any(session in x for session in range(100, 119))
        for x in bios.congresses.to_list()
    ]
]

In [422]:
def getParty(x):
    return x["congressAffiliation"]["partyAffiliation"][0]["party"]["name"][0]


temp = recentBios.loc[:, "jobPositions"].apply(
    lambda x: {
        y["congressAffiliation"]["congress"]["congressNumber"]: getParty(y)
        for y in x
        if y["congressAffiliation"]["congress"]["congressNumber"] > 99
    }
)

recentBios.loc[:, "parties"] = pd.DataFrame(
    {
        "empties": temp.apply(
            lambda x: {y: None for y in range(100, 119) if y not in x.keys()}
        ),
        "parties": temp,
    }
).apply(lambda row: dict(sorted({**row["parties"], **row["empties"]}.items())), axis=1)

temp = recentBios.loc[:, "jobPositions"].apply(
    lambda x: {
        y["congressAffiliation"]["congress"]["congressNumber"]: y["job"]["name"]
        for y in x
        if y["congressAffiliation"]["congress"]["congressNumber"] > 99
    }
)

recentBios.loc[:, "jobs"] = pd.DataFrame(
    {
        "empties": temp.apply(
            lambda x: {y: None for y in range(100, 119) if y not in x.keys()}
        ),
        "jobPositions": temp,
    }
).apply(
    lambda row: dict(sorted({**row["jobPositions"], **row["empties"]}.items())), axis=1
)

...what is that at 12395?

In [399]:
display(recentBios["profileText"][12395])

# darn it, Ben Sasse!

str12395 = (
    "a Senator from Nebraska; born in Nebraska City, Nebr., August 19, 1964; graduated"
    " Westside High School, Omaha, Nebr., 1982; B.A., biology, University of Chicago,"
    " 1986; M.B.A., marketing and finance, University of Chicago, 1991; online"
    " brokerage firm president and chief operating officer; co-owner of Chicago Cubs"
    " baseball team; unsuccessful candidate for the United States Senate in 2006;"
    " governor of Nebraska 2015-2023; appointed as a Republican to the United States"
    " Senate on January 12, 2023, to fill the vacancy caused by the resignation of"
    " Benjamin Sasse; took the oath of office on January 23, 2023."
)
recentBios.loc[12395, "profileText"] = str12395

'<!-- [if gte mso 9]><xml>\n <o:OfficeDocumentSettings>\n  <o:AllowPNG/>\n </o:OfficeDocumentSettings>\n</xml><![endif]--><!-- [if gte mso 9]><xml>\n <w:WordDocument>\n  <w:View>Normal</w:View>\n  <w:Zoom>0</w:Zoom>\n  <w:TrackMoves/>\n  <w:TrackFormatting/>\n  <w:PunctuationKerning/>\n  <w:ValidateAgainstSchemas/>\n  <w:SaveIfXMLInvalid>false</w:SaveIfXMLInvalid>\n  <w:IgnoreMixedContent>false</w:IgnoreMixedContent>\n  <w:AlwaysShowPlaceholderText>false</w:AlwaysShowPlaceholderText>\n  <w:DoNotPromoteQF/>\n  <w:LidThemeOther>EN-US</w:LidThemeOther>\n  <w:LidThemeAsian>X-NONE</w:LidThemeAsian>\n  <w:LidThemeComplexScript>X-NONE</w:LidThemeComplexScript>\n  <w:Compatibility>\n   <w:BreakWrappedTables/>\n   <w:SnapToGridInCell/>\n   <w:WrapTextWithPunct/>\n   <w:UseAsianBreakRules/>\n   <w:DontGrowAutofit/>\n   <w:SplitPgBreakAndParaMark/>\n   <w:EnableOpenTypeKerning/>\n   <w:DontFlipMirrorIndents/>\n   <w:OverrideTableStyleHps/>\n  </w:Compatibility>\n  <m:mathPr>\n   <m:mathFont m:val

In [400]:
recentBios = recentBios[
    ["givenName", "familyName", "congresses", "profileText", "jobPositions"]
]

In [401]:
def get_state_abbrev(text):
    for state in utils.state_list:
        if state.lower() in text.lower():
            return utils.state_dict[state]

    return None


recentBios.loc[:, "state"] = recentBios.profileText.apply(
    lambda x: get_state_abbrev(x.split(";")[0])
)

In [402]:
recentBios.sample(5)

,givenName,familyName,congresses,profileText,jobPositions,state
8439,Walt,Minnick,[111],A Representative from Idaho; born in Walla Wal...,"[{'job': {'name': 'Representative', 'jobType':...",ID
2902,William,Bradley,"[96, 97, 98, 99, 100, 101, 102, 103, 104]",A Senator from New Jersey; born in Crystal Cit...,"[{'job': {'name': 'Senator', 'jobType': 'Congr...",NJ
2759,Peter,Rodino,"[81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 9...",A Representative from New Jersey; born in Newa...,"[{'job': {'name': 'Representative', 'jobType':...",NJ
9532,Brad,Ellsworth,"[110, 111]","A Representative from Indiana; born in Jasper,...","[{'job': {'name': 'Representative', 'jobType':...",IN
2425,Timothy,Hutchinson,"[103, 104, 105, 106, 107]",a Representative and a Senator from Arkansas; ...,"[{'job': {'name': 'Representative', 'jobType':...",AR


### Let's confirm that there are 2 senators from each State:

Still need to rewrite the code here to account for the fact that future senators
will still show up as senators

In [403]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x != 2]

AttributeError: 'DataFrame' object has no attribute 'state'

Okay, I assume that the NE and CA anomalies are Ben Sasse retiring and Diane
Feinstein passing away, since the two of them were both members of the 118th
Congress. Let's check on that real quick:

In [404]:
recentBios.loc[
    (recentBios.familyName == "Sasse") | (recentBios.familyName == "Feinstein")
].index

Index([6720, 12640], dtype='int64')

I'm not going to drop either of them, since they were technically parts of the 118th Congress. But that still leaves the question of what's going on in VT, OK, and NC.

In [405]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: x.state == "VT"]

AttributeError: 'DataFrame' object has no attribute 'state'

Okay, looks like [Peter Welch](https://en.wikipedia.org/wiki/Peter_Welch) is missing...

In [406]:
recentBios.loc[lambda x: (x.givenName == "Peter") & (x.familyName == "Welch")]

,givenName,familyName,congresses,profileText,jobPositions,state
2107,Peter,Welch,"[110, 111, 112, 113, 114, 115, 116, 117, 118]",A Representative from Vermont; born in Springf...,"[{'job': {'name': 'Representative', 'jobType':...",VT


I guess they didn't update him in the Congressional Bioguide when he replaced Patrick Leahy? Luckily, this is an easy fix: I'll just do what I did with that massive misencoded html bio from earlier.

In [407]:
recentBios.loc[2107, "profileText"]

'A Representative from Vermont; born in Springfield, Hampden County, Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.; A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of California, Berkeley, Calif., 1973; lawyer, private practice; member of the Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the One Hundred First Congress in 1988; elected as a Democrat to the One Hundred Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023); was not a candidate for reelection to the House of Representatives but was elected as a Democrat to the United States Senate in 2022 for the term ending January 3, 2029.'

They even note that he was elected as a Democrat to the Senate at the end of his bio! If I were less lazy, I'd rewrite some code to use "elected as a x to the United States Senate," but I figure so many people get appointed that the way I'm doing it now is easier.

In [408]:
welchBio = (
    "A Senator and a Representative from Vermont; born in Springfield, Hampden County,"
    " Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.;"
    " A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of"
    " California, Berkeley, Calif., 1973; lawyer, private practice; member of the"
    " Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president"
    " pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the"
    " One Hundred First Congress in 1988; elected as a Democrat to the One Hundred"
    " Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023);"
    " was not a candidate for reelection to the House of Representatives but was"
    " elected as a Democrat to the United States Senate in 2022 for the term ending"
    " January 3, 2029."
)
recentBios.loc[2107, "profileText"] = welchBio

In [409]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

AttributeError: 'DataFrame' object has no attribute 'state'

Alright, that's VT down, now just Oklahoma and NC. My bet is that the same thing happened to Markwayne Mullin and Ted Budd.

In [410]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: (x.state == "OK") | (x.state == "NC")]

AttributeError: 'DataFrame' object has no attribute 'state'

Yep, my hunch was right.

In [411]:
display(
    recentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)
recentBios.loc[11757, "profileText"] = recentBios.loc[11757, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    recentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)

,givenName,familyName,congresses,profileText,jobPositions,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]","a Representative from Oklahoma; born in Tulsa,...","[{'job': {'name': 'Representative', 'jobType':...",OK


,givenName,familyName,congresses,profileText,jobPositions,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]",a Senator and a Representative from Oklahoma; ...,"[{'job': {'name': 'Representative', 'jobType':...",OK


In [412]:
display(
    recentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)
recentBios.loc[9656, "profileText"] = recentBios.loc[9656, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    recentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)

,givenName,familyName,congresses,profileText,jobPositions,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Representative from North Carolina; born in ...,"[{'job': {'name': 'Representative', 'jobType':...",NC


,givenName,familyName,congresses,profileText,jobPositions,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Senator and a Representative from North Caro...,"[{'job': {'name': 'Representative', 'jobType':...",NC


In [413]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

AttributeError: 'DataFrame' object has no attribute 'state'

Alright, now we can easily flag everyone who was a senator in the 118th congress! Let's make an indicator real quick:

In [414]:
currentBios["senator"] = (
    currentBios.profileText.str.lower().str.contains("a senator from")
    | currentBios.profileText.str.lower().str.contains("a senator and a represent")
    | currentBios.profileText.str.lower().str.contains("a senator and represent")
).astype(bool)

/var/folders/j9/cgws7l217tl7gwgvk9x0ckf80000gn/T/ipykernel_63474/820753479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currentBios["senator"] = (


### Checking representatives

In [415]:
currentBios.senator.apply(lambda x: not x).sum()

np.int64(444)

Okay, this is what I expected! There are, after all, [6 non-voting members of the House of Representatives](https://en.wikipedia.org/wiki/Non-voting_members_of_the_United_States_House_of_Representatives), and 435 = 441 - 6. I don't really care about the non-voting members, though (no offense), so I'll be dropping them (as well as making sure each state has the correct number of reps).

I might want to write a more robust script to handle Congresses before 1960 (i.e. more robust "was this state a state at the time" handling), but that isn't a problem right now.

In [416]:
currentBios.loc[currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()]

AttributeError: 'DataFrame' object has no attribute 'state'

In [417]:
currentBios.drop(
    currentBios.loc[
        currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()
    ].index,
    inplace=True,
)

AttributeError: 'DataFrame' object has no attribute 'state'

In [418]:
reps = {
    get_state_abbrev(k): v
    for k, v in zip(utils.number_of_reps.keys(), utils.number_of_reps.values())
}

In [419]:
# confirming that every state has the correct number of reps

[
    state
    for state in currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .index
    if currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .loc[state]
    != reps[state]
]

AttributeError: 'DataFrame' object has no attribute 'state'

In [420]:
currentBios["congressLabels"] = currentBios.apply(
    lambda row: f"{row.givenName} {row.familyName} ({row.party}-{row.state})",
    axis=1,
)

AttributeError: 'Series' object has no attribute 'party'

### Applying categories

In [472]:
import json
import re

from anthropic import Anthropic
from anthropic.types.beta.message_create_params import (
    MessageCreateParamsNonStreaming,
)
from anthropic.types.beta.messages.batch_create_params import Request
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

SYSTEM_PROMPT = """You're a data analyst, tasked with analyzing whether certain
career milestones pertain based upon a biography you're given. I will give you
a biography, a list of JSON keys, and a brief description for each key. Your
job will be to read through the biography, then, for each JSON key, return
either the part of the biography that makes you think that key pertains or the
boolean `false`. If there's more than one part of the biography that makes you
think the key pertains, return only the first part of the biography that makes
you think the key pertains.

For example, with JSON keys
    ['communityCollege', 'eliteCollege', 'publicCollege', 'privateCollege', 'noBA']
and a biography that starts:
    "a Representative from Missouri; born in Baytown, Harris County, Tex., October 4, 1963; graduated from Ross S. Sterling High School, Baytown, Tex., 1982; attended Lee College, Baytown, Tex.; attended Southwest Texas State College, San Marcos, Tex.; attended University of Texas, Austin, Tex.; television news anchor;..."

Your response should be as follows:
{
    "communityCollege": "Lee College, Baytown, Tex.",
    "eliteCollege": false,
    "publicCollege": "attended Southwest Texas State College, San Marcos, Tex.",
    "privateCollege": false,
    "noBA": false
}
"""

KEYS_AND_EXPLANATIONS = """
    'communityCollege', # does this person have an associates degree and/or attend a community college?
    'eliteCollege', # did this person complete their UNDERGRAD education at an Ivy League School, Stanford, UChicago, Duke, or MIT? say False if it's a grad degree
    'publicCollege', # undergrad at public school
    'privateCollege', # ud
    'noBA', # no bachelors degree
    'lawSchool', # JD, LLM, or any other law degree
    'medDegree', # include DOs and nursing degrees, too
    'doctorate',
    'mba',
    'otherMasters',
    'military',
    'lawyerPrivatePractice',
    'publicLawyerOrJudge',
    'prevUnsuccessfulCandidate',
    'partyPolitics', # did they ever work for the local/state/national branch of Democratic/Republican/third party? don't include "elected as a Democrat" or "elected as a Republican"
    'sciOrEng', # ever have a job
    'religion',
    'healthcare',
    'blueCollarOrServiceJob',
    'education', # only include people who were teachers / professors / administrators
    'nonprofitsAndUnions',
    'sports', # both players and coaches
    'media',
    'realEstate',
    'lawEnforcement',
    'farmingOrRanching',
    'businessOrManagement',
    'lobbyingOrPacs',
    'localGov',
    'stateLeg',
    'electedFedOrStateOffice', # don't include being in the state legislature, DO NOT include being elected to congress. CA high speed rail authority isn't elected.
    'nonElectedGovJob' # everything from staffer/aide in a legislature to being an appointed official
"""

message_batch = client.beta.messages.batches.create(
    requests=[
        Request(
            custom_id=f"{i}",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-5-sonnet-20241022",
                max_tokens=512,
                temperature=0.1,
                system=SYSTEM_PROMPT,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": KEYS_AND_EXPLANATIONS,
                            },
                            {
                                "type": "text",
                                "text": "Here's the biography:\n\n" + x["profileText"],
                            },
                        ],
                    },
                    {
                        "role": "assistant",
                        "content": [
                            {
                                "type": "text",
                                "text": '{"communityCollege":',
                            },
                        ],
                    },
                ],
            ),
        )
        for i, x in recentBios.iterrows()
    ]
)

In [473]:
batch_id = message_batch.id

In [474]:
import time

while True:
    message_batch = client.beta.messages.batches.retrieve(batch_id)
    if message_batch.processing_status == "ended":
        break

    print(f"Batch {batch_id} is still processing...")
    time.sleep(60)
print(message_batch)

Batch msgbatch_01KHU5j68SjBcd2KVApCYd9s is still processing...
Batch msgbatch_01KHU5j68SjBcd2KVApCYd9s is still processing...
Batch msgbatch_01KHU5j68SjBcd2KVApCYd9s is still processing...
BetaMessageBatch(id='msgbatch_01KHU5j68SjBcd2KVApCYd9s', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2024, 12, 1, 5, 13, 41, 927231, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2024, 12, 1, 5, 16, 2, 891574, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2024, 12, 2, 5, 13, 41, 927231, tzinfo=datetime.timezone.utc), processing_status='ended', request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=0, succeeded=1939), results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01KHU5j68SjBcd2KVApCYd9s/results', type='message_batch')


In [475]:
def remove_comments(text):
    pattern = r"(\\|#).*\n"
    text = re.sub(pattern, "", text)

    return text


flags = []

for result in client.beta.messages.batches.results(
    batch_id,
):
    flags.append(
        json.loads(
            '{"communityCollege": '
            + remove_comments(result.result.message.content[0].text)
        )
    )

In [476]:
df = pd.merge(
    recentBios,
    pd.DataFrame(flags, index=recentBios.index),
    left_index=True,
    right_index=True,
)

In [ ]:
import ast

df = pd.read_csv("../../data/processed-bios.csv", index_col=0).drop(
    columns=["Unnamed: 3"]
)


df.loc[:, "congresses"] = df.loc[:, "congresses"].apply(ast.literal_eval)
df.loc[:, "jobPositions"] = df.loc[:, "jobPositions"].apply(ast.literal_eval)

df.loc[:, "parties"] = (
    df.loc[:, "parties"]
    .apply(ast.literal_eval)
    .apply(lambda d: {int(k): v for k, v in d.items()})
)

df.loc[:, "jobs"] = (
    df.loc[:, "jobs"]
    .apply(ast.literal_eval)
    .apply(lambda d: {int(k): v for k, v in d.items()})
)

df = df.replace("FALSE", False)
df.to_json("../../data/processed-bios.json", orient="index")

In [522]:
df = pd.read_json("../../data/processed-bios.json").T

In [ ]:
df.drop(df[df["state"].isna()].index).to_json(
    "../../data/processed-bios.json", orient="index"
)

In [553]:
df = pd.read_json("../../data/processed-bios.json").T

In [ ]:
df.loc[:, "givenName"] = bios.givenName[df.index]
df.loc[:, "familyName"] = bios.familyName[df.index]

df.loc[:, "unaccentedFamilyName"] = bios["unaccentedFamilyName"][df.index]
df.loc[:, "unaccentedGivenName"] = bios["unaccentedGivenName"][df.index]

In [567]:
df.to_json("../../data/processed-bios.json", orient="index")

### (deprecated) Applying categories

Here, I put the csv into Google Sheets and did a whole lot more work on it — basically all manual, which wasn't fun. Turns out it's tough to hard-code which things ought to count as which categories. I tried some GPT-4 api calls, but I didn't end up doing too well. Good news is that, after poking around there, I can do it more efficiently here below.

The labels below are almost entirely the originals from the [New York Times piece](https://www.nytimes.com/interactive/2019/01/26/opinion/sunday/paths-to-congress.html) except that I added
- previously unsuccessful candidate for office,
- more specific master's degrees,
- previous experience in party politics.

I think I was also more generous with what counts as a medical degree, including nursing and osteopathic degrees.

You can see what I checked for each thing by examining the flags I've added for each category in in the `utils` module, though. 

In [25]:
def splitAndTest(profileText, contains, doesntContain=[]):
    bioItems = profileText.lower().split(";")
    if doesntContain:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    if not any(dc.lower() in bioItem for dc in doesntContain):
                        return bioItem
                    else:
                        continue
        return False

    else:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    return bioItem
        return False

In [26]:
currentBios["communityCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.communityCollege, ["board", "professor", "instructor", "president"]
    )
)

In [27]:
currentBios["eliteCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.eliteCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
        ],
    )
)

In [28]:
currentBios["publicCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.publicCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            # "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [29]:
currentBios["privateCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.privateCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [30]:
currentBios["noBA"] = currentBios.apply(
    lambda x: (
        (not x["communityCollege"])
        & (not x["eliteCollege"])
        & (not x["publicCollege"])
        & (not x["privateCollege"])
    ),
    axis=1,
)

In [31]:
currentBios["lawSchool"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawSchool)
)

In [32]:
currentBios["medDegree"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.medDegree)
)

In [33]:
currentBios["doctorate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.doctorate)
)

In [34]:
currentBios["mba"] = currentBios.profileText.apply(lambda x: splitAndTest(x, utils.mba))

In [35]:
currentBios["otherMasters"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.otherMasters)
)

In [36]:
currentBios["military"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.military, ["academy", "college"])
)

In [37]:
currentBios["lawyerPrivatePractice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawyerPrivatePractice)
)
currentBios["lawyerPrivatePractice"] = [
    (
        currentBios.lawyerPrivatePractice.iloc[i]
        if currentBios.lawyerPrivatePractice.iloc[i]
        else (
            "lawyer"
            if any(
                [
                    (
                        bioItem
                        if (bioItem.strip() == "lawyer")
                        | (bioItem.strip() == "attorney")
                        else False
                    )
                    for bioItem in currentBios.profileText.iloc[i].lower().split(";")
                ]
            )
            else False
        )
    )
    for i in range(len(currentBios))
]

In [38]:
currentBios["publicLawyerOrJudge"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.publicLawyerOrJudge, ["college counselor", "one of the managers"]
    )
)

In [39]:
currentBios["prevUnsuccessfulCandidate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.prevUnsuccessfulCandidate)
)

In [40]:
currentBios["partyPolitics"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.partyPolitics + ["democratic", "republican"],
        [
            "elected as a republican",
            "united states",
            "unsuccessful",
            "independent",
            "select committee on the strategic competition",
            "governor of maine",
        ],
    )
)

In [41]:
currentBios["sciOrEng"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sciOrEng, ["b.s.", "m.s."])
)

In [42]:
currentBios["religion"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.religion, ["b.s.", "m.s."])
)

In [43]:
currentBios["healthcare"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.healthcare, ["b.s.", "m.s.", "m.d."])
)

In [44]:
currentBios["blueCollarOrServiceJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.blueCollarOrServiceJob, ["academy"])
)

In [45]:
currentBios["education"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.education, ["elected as a republican"])
)

In [46]:
currentBios["nonprofitsAndUnions"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonprofitsAndUnions,
        [
            "credit union",
            "union, n.j.",
            "union county, n.j.",
            "union, miss.",
            "union city",
            "union theological seminary",
            "ph.d., union institute",
        ],
    )
)

In [47]:
currentBios["sports"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sports, ["coachella valley high school"])
)

In [48]:
currentBios["media"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.media, ["anchorage", "authority", "authorized", "speechwriter"]
    )
)

In [49]:
currentBios["realEstate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.realEstate)
)

In [50]:
currentBios["lawEnforcement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawEnforcement, ["attorney"])
)

In [51]:
currentBios["farmingOrRanching"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.farmingOrRanching, ["chair, committee on agriculture", "administrator"]
    )
)

In [52]:
currentBios["businessOrManagement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.businessOrManagement,
        [
            "campaign consultant",
            "central intelligence agency",
            "national nuclear security administration",
            "fairbanks",
            "committee on banking",
            "b.a., business administration",
            "county executive",
            "red bank, monmouth county",
            "executive office of the president",
            "committee on small business",
            "osceola",
            "nonprofit",
            "non-profit",
            "small business administration",
            "federal coordinator for gulf coast rebuilding",
            "m.b.a.",
            "m.p.a",
            "bankruptcy court",
            "business administration",
            "secretary of state",
            "government analyst",
            "california democratic state central executive committee",
            "executive director, human services",
            "jefferson county",
            "business regulation",
            "commission",
            "department of regulatory agencies",
            "nevada system of higher education",
            "republican state committee",
            "workers' compensation board",
            "afl-cio",
        ],
    )
)

In [53]:
currentBios["lobbyingOrPacs"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lobbyingOrActivism)
)

In [54]:
currentBios["localGov"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.localGov, ["graduated from", "b.a.", "unsuccessful candidate for"]
    )
)

In [55]:
currentBios["stateLeg"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.stateLeg,
        ["legislative assistant", "aide", "staff", "counsel", "fellow", "unsuccessful"],
    )
)

In [56]:
currentBios["electedFedOrStateOffice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.electedFedOrStateOffice,
        [
            "appointed",
            "assistant u.s. secretary of state",
            "inland empire regional office",
            "counsel",
            "advisor",
            "unsuccessful",
            "lost general election",
        ],
    )
)

In [57]:
currentBios["nonElectedGovJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonElectedGovJob + ["advisor"],
        [
            "county",
            "college",
            "development advisor",
            "emergency 9-1-1 dispatcher",
            "nonprofit organization",
            "n.y. council",
            "graduated",
            "postal inspector",
            "inland empire regional office",
            "chapel hill, n.c., police department",
            "long beach, calif",
        ],
    )
)